In [ ]:
import sys
import os
import re
import numpy as np
import pandas as pd
import scipy

import mne

from extract_scalar import *

In [ ]:
# set different path to load and save data
ds_root='/volumes/hyijie_psy/CPP/data_mid-level_2/data_raw'
tgt_dir='..'

dir_preprocess_raw_mem=os.path.join(tgt_dir,'data','preprocessData','raw','mem')
if not os.path.exists(dir_preprocess_raw_mem):
    os.makedirs(dir_preprocess_raw_mem)

dir_preprocess_raw_perc=os.path.join(tgt_dir,'data','preprocessData','raw','perc')
if not os.path.exists(dir_preprocess_raw_perc):
    os.makedirs(dir_preprocess_raw_perc)

result_dir_joint_modeling_mem=os.path.join(tgt_dir,'results','joint-modeling_mem')
if not os.path.exists(result_dir_joint_modeling_mem):
    os.makedirs(result_dir_joint_modeling_mem)

result_dir_joint_modeling_perc=os.path.join(tgt_dir,'results','joint-modeling_perc')
if not os.path.exists(result_dir_joint_modeling_perc):
    os.makedirs(result_dir_joint_modeling_perc)


In [ ]:
sampleRate = 500

# memory data
sub_dir = [f'ACC{i:03}' for i in range(14,37) ]
for sub in sub_dir:
    raw = scipy.io.loadmat(os.path.join(ds_root,sub,sub+'.mat'))
    raw_stimuluLock = raw['trialdata'][0]['trial'][0]

    behavior = scipy.io.loadmat(os.path.join(ds_root, sub ,'events.mat'))['events'][['mode', 'type', 'rt']]
    behavior_df = pd.DataFrame({
        'mode': [extract_scalar(x) for x in behavior['mode'][0]], 
        'type': [extract_scalar(x) for x in behavior['type'][0]],  
        'rt': [extract_scalar(x) for x in behavior['rt'][0]]       
    })

    response_df = behavior_df[behavior_df['type'] == 'resp']
    response_df = response_df.reset_index(drop=True)
    response4mem = response_df[response_df['mode'] == 'mem']['rt']

    data_responseLock_mem = np.empty((1,len(response4mem)), dtype=object) #保存为（1，试次数）

    for i, (index, rt_value) in enumerate(response4mem.items()):
        data_epoch = raw_stimuluLock[0,index][:, (-int(1.3*sampleRate)) :(-int(0.1*sampleRate-1))] #减1是左闭右闭
        data_responseLock_mem[0,i] = data_epoch

    # save eeg data
    data_responseLock_mem_save={
        "data_responseLock_mem":data_responseLock_mem
    }
    save_path=os.path.join(dir_preprocess_raw_mem,'responseLock_'+sub+'mem.mat')
    scipy.io.savemat(save_path, data_responseLock_mem_save)           

In [ ]:
sampleRate = 500
# perception data
sub_dir = [f'ACC{i:03}' for i in range(14,37) ]
for sub in sub_dir:
    raw = scipy.io.loadmat(os.path.join(ds_root,sub,sub+'.mat'))
    raw_stimuluLock = raw['trialdata'][0]['trial'][0]

    behavior = scipy.io.loadmat(os.path.join(ds_root, sub ,'events.mat'))['events'][['mode', 'type', 'rt']]
    behavior_df = pd.DataFrame({
        'mode': [extract_scalar(x) for x in behavior['mode'][0]], 
        'type': [extract_scalar(x) for x in behavior['type'][0]],  
        'rt': [extract_scalar(x) for x in behavior['rt'][0]]       
    })

    response_df = behavior_df[behavior_df['type'] == 'resp']
    response_df = response_df.reset_index(drop=True)
    response4perc = response_df[response_df['mode'] == 'perc']['rt']

    data_responseLock_perc = np.empty((1,len(response4perc)), dtype=object) #保存为（1，试次数）

    for i, (index, rt_value) in enumerate(response4perc.items()):
        data_epoch = raw_stimuluLock[0,index][:, (-int(1.3*sampleRate)) :(-int(0.1*sampleRate-1))] #减1是左闭右闭
        data_responseLock_perc[0,i] = data_epoch

    # save eeg data
    data_responseLock_perc_save={
        "data_responseLock_perc":data_responseLock_perc
    }
    save_path=os.path.join(dir_preprocess_raw_perc,'responseLock_'+sub+'perc.mat')
    scipy.io.savemat(save_path, data_responseLock_perc)     